In [2]:
from skimage import transform, filters, exposure
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.cross_validation import train_test_split
from keras.layers.convolutional import Conv2D

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import os
import glob
import pandas as pd
import math
from PIL import Image
import numpy as np
from scipy.misc import imread, imsave, imresize
from natsort import natsorted
from scipy.misc import imread
import random
random.seed( 42 ) 

# Path of data files
path = "/home/ivan/python/julia/data"
 
# Input image dimensions
img_rows, img_cols = 32, 32
 
 

### Images preprocessing ###

for setType in ["test","train"]:
    # We have to make sure files are sorted according to labels, even if they don't have trailing zeros
    files = natsorted(glob.glob(path + "/"+setType+"new/*"))
    """
    if setType == 'test':
        X_test = np.array([np.array(Image.open(fname)) for fname in files])
    else:
        X_train = np.array([np.array(Image.open(fname)) for fname in files])
    """
    if setType == 'test':
        X_test = np.array([np.array(imread(fname, flatten=True)) for fname in files])
    else:
        X_train = np.array([np.array(imread(fname, flatten=True)) for fname in files])
X_test=X_test[:,:,:,np.newaxis]
X_train=X_train[:,:,:,np.newaxis]
X_train /= 255
X_test /= 255
X_test.shape
    

(6220, 20, 20, 1)

In [4]:
def label2int(ch):
    asciiVal = ord(ch)
    if(asciiVal<=57): #0-9
        asciiVal-=48
    elif(asciiVal<=90): #A-Z
        asciiVal-=55
    else: #a-z
        asciiVal-=61
    return asciiVal
    
def int2label(i):
    if(i<=9): #0-9
        i+=48
    elif(i<=35): #A-Z
        i+=55
    else: #a-z
        i+=61
    return chr(i)
# Load labels
y_train = pd.read_csv("/home/ivan/python/julia/trainLabels.csv").values[:,1] #Keep only label
 
# Convert labels to one-hot vectors
Y_train = np.zeros((y_train.shape[0], len(np.unique(y_train))))
 
for i in range(y_train.shape[0]):
    Y_train[i][label2int(y_train[i])] = 1 # One-hot
Y_train_all = Y_train.copy()
X_train_all = X_train.copy()

In [5]:
X_train, X_val, Y_train, Y_val = \
        train_test_split(X_train_all, Y_train_all, test_size=0.2, stratify=np.argmax(Y_train_all, axis=1))

In [6]:
nb_classes = 62 # A-Z, a-z and 0-9
nb_epoch = 500
img_rows, img_cols = 20, 20
model = Sequential()

#model.add(Convolution2D(3, 3, 128,  border_mode='same', init='he_normal', activation = 'relu', input_shape=(1, img_rows, img_cols)))
model.add(Conv2D(3, (3, 32), kernel_initializer="he_normal", activation="relu", input_shape=(img_rows, img_cols, 1), padding="same"))
model.add(Conv2D(3, (3, 32), kernel_initializer="he_normal", activation="relu", padding="same"))

model.add(MaxPooling2D(data_format="channels_last", pool_size=(2, 2)))

model.add(Conv2D(3, (3, 64), kernel_initializer="he_normal", activation="relu", padding="same"))
#model.add(Conv2D(3, (3, 256), kernel_initializer="he_normal", activation="relu", padding="same"))

model.add(MaxPooling2D(data_format="channels_last", pool_size=(2, 2)))

model.add(Conv2D(3, (3, 128), kernel_initializer="he_normal", activation="relu", padding="same"))
model.add(Conv2D(3, (3, 256), kernel_initializer="he_normal", activation="relu", padding="same"))
#model.add(Conv2D(3, (3, 512), kernel_initializer="he_normal", activation="relu", padding="same"))

model.add(MaxPooling2D(data_format="channels_last", pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(2048, kernel_initializer="he_normal", activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2048, kernel_initializer="he_normal", activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, kernel_initializer="he_normal", activation="softmax"))

In [7]:
### LEARNING ###
 
# First, use AdaDelta for some epochs because AdaMax gets stuck
model.compile(loss='categorical_crossentropy', 
              optimizer='adadelta',  
              metrics=["accuracy"])

In [8]:
# 20 epochs is sufficient
model.fit(X_train, Y_train, batch_size=128,
                    #nb_epoch=20, 
                    epochs=20,
                    validation_data=(X_val, Y_val),
                    verbose=1)

Train on 5026 samples, validate on 1257 samples
Epoch 1/20
5026/5026 [==============================] - 19s - loss: 4.0025 - acc: 0.0631 - val_loss: 3.7980 - val_acc: 0.0573
Epoch 2/20
5026/5026 [==============================] - 19s - loss: 3.8017 - acc: 0.0657 - val_loss: 3.7745 - val_acc: 0.0676
Epoch 3/20
5026/5026 [==============================] - 19s - loss: 3.7880 - acc: 0.0682 - val_loss: 3.7750 - val_acc: 0.0732
Epoch 4/20
5026/5026 [==============================] - 19s - loss: 3.7783 - acc: 0.0700 - val_loss: 3.7505 - val_acc: 0.0732
Epoch 5/20
5026/5026 [==============================] - 19s - loss: 3.7653 - acc: 0.0736 - val_loss: 3.7667 - val_acc: 0.0732
Epoch 6/20
5026/5026 [==============================] - 19s - loss: 3.7333 - acc: 0.0820 - val_loss: 3.6422 - val_acc: 0.0971
Epoch 7/20
5026/5026 [==============================] - 19s - loss: 3.6022 - acc: 0.1238 - val_loss: 3.4846 - val_acc: 0.1671
Epoch 8/20
5026/5026 [==============================] - 19s - loss: 3.

In [9]:
# Parametrize the image augmentation class
datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    shear_range = 0.4,
    zoom_range = 0.3,                    
    channel_shift_range = 0.1)

In [10]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adamax',  
              metrics=["accuracy"])

# We want to keep the best model. This callback will store 
# in a file the weights of the model with the highest validation accuracy  
saveBestModel = ModelCheckpoint("best.kerasModelWeights", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True)

# Make the model learn using the image generator
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),
                samples_per_epoch=len(X_train),
                epochs=500, 
                validation_data=(X_val, Y_val),
                callbacks=[saveBestModel],
                verbose=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., callbacks=[<keras.ca..., steps_per_epoch=39, validation_data=(array([[[..., epochs=500, verbose=1)`
  from ipykernel import kernelapp as app


Epoch 1/500
39/39 [==============================] - 19s - loss: 3.2680 - acc: 0.1943 - val_loss: 2.3852 - val_acc: 0.3994
Epoch 2/500
39/39 [==============================] - 18s - loss: 3.1383 - acc: 0.2171 - val_loss: 2.2871 - val_acc: 0.4113
Epoch 3/500
39/39 [==============================] - 18s - loss: 3.0535 - acc: 0.2297 - val_loss: 2.2323 - val_acc: 0.4304
Epoch 4/500
39/39 [==============================] - 18s - loss: 3.0316 - acc: 0.2303 - val_loss: 2.2450 - val_acc: 0.4256
Epoch 5/500
39/39 [==============================] - 18s - loss: 3.0086 - acc: 0.2438 - val_loss: 2.1663 - val_acc: 0.4455
Epoch 6/500
39/39 [==============================] - 18s - loss: 2.9885 - acc: 0.2497 - val_loss: 2.1938 - val_acc: 0.4399
Epoch 7/500
39/39 [==============================] - 21s - loss: 2.9487 - acc: 0.2484 - val_loss: 2.1847 - val_acc: 0.4375
Epoch 8/500
39/39 [==============================] - 19s - loss: 2.9660 - acc: 0.2418 - val_loss: 2.1213 - val_acc: 0.4519
Epoch 9/500
39/3

39/39 [==============================] - 18s - loss: 2.6293 - acc: 0.3173 - val_loss: 1.8361 - val_acc: 0.4869
Epoch 33/500
39/39 [==============================] - 18s - loss: 2.6276 - acc: 0.3247 - val_loss: 1.8277 - val_acc: 0.4909
Epoch 34/500
39/39 [==============================] - 18s - loss: 2.6271 - acc: 0.3198 - val_loss: 1.8285 - val_acc: 0.4980
Epoch 35/500
39/39 [==============================] - 18s - loss: 2.6197 - acc: 0.3129 - val_loss: 1.8054 - val_acc: 0.5084
Epoch 36/500
39/39 [==============================] - 18s - loss: 2.6179 - acc: 0.3159 - val_loss: 1.8548 - val_acc: 0.4877
Epoch 37/500
39/39 [==============================] - 18s - loss: 2.5893 - acc: 0.3233 - val_loss: 1.7849 - val_acc: 0.5123
Epoch 38/500
39/39 [==============================] - 18s - loss: 2.5819 - acc: 0.3226 - val_loss: 1.7965 - val_acc: 0.4988
Epoch 39/500
39/39 [==============================] - 18s - loss: 2.6174 - acc: 0.3155 - val_loss: 1.8149 - val_acc: 0.4980
Epoch 40/500
39/39 [=

39/39 [==============================] - 18s - loss: 2.5065 - acc: 0.3388 - val_loss: 1.6881 - val_acc: 0.5274
Epoch 65/500
39/39 [==============================] - 18s - loss: 2.4151 - acc: 0.3486 - val_loss: 1.6660 - val_acc: 0.5298
Epoch 66/500
39/39 [==============================] - 18s - loss: 2.4564 - acc: 0.3492 - val_loss: 1.6712 - val_acc: 0.5354
Epoch 67/500
39/39 [==============================] - 18s - loss: 2.4791 - acc: 0.3490 - val_loss: 1.6702 - val_acc: 0.5434
Epoch 68/500
39/39 [==============================] - 18s - loss: 2.4390 - acc: 0.3507 - val_loss: 1.6865 - val_acc: 0.5330
Epoch 69/500
39/39 [==============================] - 18s - loss: 2.4570 - acc: 0.3486 - val_loss: 1.6678 - val_acc: 0.5370
Epoch 70/500
39/39 [==============================] - 18s - loss: 2.4208 - acc: 0.3525 - val_loss: 1.6437 - val_acc: 0.5465
Epoch 71/500
39/39 [==============================] - 18s - loss: 2.4337 - acc: 0.3512 - val_loss: 1.6584 - val_acc: 0.5386
Epoch 72/500
39/39 [=

39/39 [==============================] - 18s - loss: 2.3618 - acc: 0.3637 - val_loss: 1.5690 - val_acc: 0.5577
Epoch 98/500
39/39 [==============================] - 18s - loss: 2.3267 - acc: 0.3698 - val_loss: 1.5987 - val_acc: 0.5513
Epoch 99/500
39/39 [==============================] - 18s - loss: 2.3461 - acc: 0.3692 - val_loss: 1.6034 - val_acc: 0.5529
Epoch 100/500
39/39 [==============================] - 19s - loss: 2.3475 - acc: 0.3713 - val_loss: 1.5607 - val_acc: 0.5632
Epoch 101/500
39/39 [==============================] - 18s - loss: 2.3029 - acc: 0.3856 - val_loss: 1.5550 - val_acc: 0.5728
Epoch 102/500
39/39 [==============================] - 18s - loss: 2.3410 - acc: 0.3645 - val_loss: 1.5732 - val_acc: 0.5609
Epoch 103/500
39/39 [==============================] - 19s - loss: 2.3550 - acc: 0.3552 - val_loss: 1.5707 - val_acc: 0.5569
Epoch 104/500
39/39 [==============================] - 18s - loss: 2.3001 - acc: 0.3800 - val_loss: 1.5862 - val_acc: 0.5513
Epoch 105/500
39

...

39/39 [==============================] - 19s - loss: 1.9999 - acc: 0.4288 - val_loss: 1.3532 - val_acc: 0.6245
Epoch 365/500
39/39 [==============================] - 19s - loss: 1.9637 - acc: 0.4625 - val_loss: 1.3428 - val_acc: 0.6253
Epoch 366/500
39/39 [==============================] - 19s - loss: 2.0089 - acc: 0.4374 - val_loss: 1.3597 - val_acc: 0.6173
Epoch 367/500
39/39 [==============================] - 19s - loss: 1.9871 - acc: 0.4442 - val_loss: 1.3538 - val_acc: 0.6261
Epoch 368/500
39/39 [==============================] - 19s - loss: 1.9992 - acc: 0.4458 - val_loss: 1.3704 - val_acc: 0.6173
Epoch 369/500
39/39 [==============================] - 19s - loss: 2.0048 - acc: 0.4413 - val_loss: 1.3610 - val_acc: 0.6221
Epoch 370/500
39/39 [==============================] - 19s - loss: 1.9670 - acc: 0.4527 - val_loss: 1.3504 - val_acc: 0.6285
Epoch 371/500
39/39 [==============================] - 19s - loss: 1.9674 - acc: 0.4453 - val_loss: 1.3491 - val_acc: 0.6309
Epoch 372/500


39/39 [==============================] - 19s - loss: 1.9709 - acc: 0.4479 - val_loss: 1.3515 - val_acc: 0.6333
Epoch 399/500
39/39 [==============================] - 19s - loss: 1.9401 - acc: 0.4529 - val_loss: 1.3909 - val_acc: 0.6150
Epoch 400/500
39/39 [==============================] - 19s - loss: 1.9480 - acc: 0.4514 - val_loss: 1.3434 - val_acc: 0.6229
Epoch 401/500
39/39 [==============================] - 19s - loss: 1.9588 - acc: 0.4571 - val_loss: 1.3552 - val_acc: 0.6150
Epoch 402/500
39/39 [==============================] - 19s - loss: 1.9701 - acc: 0.4428 - val_loss: 1.3805 - val_acc: 0.6134
Epoch 403/500
39/39 [==============================] - 19s - loss: 1.9859 - acc: 0.4479 - val_loss: 1.3396 - val_acc: 0.6245
Epoch 404/500
39/39 [==============================] - 19s - loss: 1.9872 - acc: 0.4490 - val_loss: 1.3495 - val_acc: 0.6293
Epoch 405/500
39/39 [==============================] - 19s - loss: 1.9525 - acc: 0.4553 - val_loss: 1.3607 - val_acc: 0.6158
Epoch 406/500


39/39 [==============================] - 19s - loss: 1.9523 - acc: 0.4435 - val_loss: 1.3351 - val_acc: 0.6396
Epoch 432/500
39/39 [==============================] - 19s - loss: 1.9152 - acc: 0.4707 - val_loss: 1.3393 - val_acc: 0.6309
Epoch 433/500
39/39 [==============================] - 19s - loss: 1.9648 - acc: 0.4434 - val_loss: 1.3450 - val_acc: 0.6229
Epoch 434/500
39/39 [==============================] - 19s - loss: 1.9249 - acc: 0.4549 - val_loss: 1.3300 - val_acc: 0.6325
Epoch 435/500
39/39 [==============================] - 19s - loss: 1.9395 - acc: 0.4551 - val_loss: 1.3428 - val_acc: 0.6237
Epoch 436/500
39/39 [==============================] - 19s - loss: 1.9786 - acc: 0.4508 - val_loss: 1.3496 - val_acc: 0.6301
Epoch 437/500
39/39 [==============================] - 19s - loss: 1.9363 - acc: 0.4589 - val_loss: 1.3222 - val_acc: 0.6356
Epoch 438/500
39/39 [==============================] - 19s - loss: 1.9544 - acc: 0.4517 - val_loss: 1.3284 - val_acc: 0.6364
Epoch 439/500


39/39 [==============================] - 19s - loss: 1.9348 - acc: 0.4536 - val_loss: 1.2961 - val_acc: 0.6317
Epoch 466/500
39/39 [==============================] - 19s - loss: 1.9108 - acc: 0.4554 - val_loss: 1.3098 - val_acc: 0.6317
Epoch 467/500
39/39 [==============================] - 19s - loss: 1.9463 - acc: 0.4538 - val_loss: 1.3302 - val_acc: 0.6309
Epoch 468/500
39/39 [==============================] - 19s - loss: 1.8921 - acc: 0.4608 - val_loss: 1.3260 - val_acc: 0.6356
Epoch 469/500
39/39 [==============================] - 19s - loss: 1.9236 - acc: 0.4594 - val_loss: 1.3060 - val_acc: 0.6348
Epoch 470/500
39/39 [==============================] - 19s - loss: 1.9673 - acc: 0.4592 - val_loss: 1.3228 - val_acc: 0.6317
Epoch 471/500
39/39 [==============================] - 19s - loss: 1.9486 - acc: 0.4536 - val_loss: 1.3230 - val_acc: 0.6412
Epoch 472/500
39/39 [==============================] - 19s - loss: 1.9332 - acc: 0.4571 - val_loss: 1.3507 - val_acc: 0.6293
Epoch 473/500


39/39 [==============================] - 19s - loss: 1.9410 - acc: 0.4570 - val_loss: 1.3401 - val_acc: 0.6269
Epoch 499/500
39/39 [==============================] - 19s - loss: 1.9160 - acc: 0.4552 - val_loss: 1.3462 - val_acc: 0.6340
Epoch 500/500
39/39 [==============================] - 19s - loss: 1.8989 - acc: 0.4723 - val_loss: 1.2977 - val_acc: 0.6340


In [12]:
# Predict the class (give the index in the one-hot vector of the most probable class)
Y_test_pred = model.predict_classes(X_test)

# Translate integers to character labels
vInt2label = np.vectorize(int2label)
Y_test_pred = vInt2label(Y_test_pred)

# Save the predicitions in Kaggle format
np.savetxt(path+"/CNN_pred.csv", np.c_[range(6284,len(Y_test_pred)+6284),Y_test_pred], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')



6208/6220 [============================>.] - ETA: 0s